In [30]:
import pickle
import gensim

In [31]:
#reading the variables from file : dictionary and tdf/corpus
with open(r"C:\Users\kanis\Major-Project\Tokens\tkn01.pkl","rb") as fp:
    data = pickle.load(fp)

dictionary = data[0]
termdocfreq = data[1]
finallemma = data[2]
feedtodict = data[3]

In [32]:
from pprint import pprint

# number of topics
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=termdocfreq,
                                       id2word=dictionary,
                                       num_topics=num_topics,random_state=100, chunksize=100, passes=10, per_word_topics=True)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[termdocfreq]

[(0,
  '0.037*"word" + 0.031*"language" + 0.024*"learning" + 0.021*"learn" + '
  '0.015*"help" + 0.014*"thing" + 0.013*"one" + 0.012*"reading" + 0.011*"day" '
  '+ 0.011*"important"'),
 (1,
  '0.035*"language" + 0.033*"thing" + 0.029*"watch" + 0.029*"show" + '
  '0.029*"movie" + 0.025*"reading" + 0.025*"phrase" + 0.023*"trying" + '
  '0.022*"like" + 0.016*"Possibly"'),
 (2,
  '0.038*"word" + 0.027*"language" + 0.023*"learning" + 0.020*"learn" + '
  '0.017*"help" + 0.013*"important" + 0.012*"thing" + 0.012*"phrase" + '
  '0.011*"reading" + 0.011*"version"'),
 (3,
  '0.033*"language" + 0.027*"learning" + 0.026*"learn" + 0.024*"phrase" + '
  '0.020*"important" + 0.020*"word" + 0.018*"thing" + 0.018*"reading" + '
  '0.015*"understand" + 0.014*"sentence"'),
 (4,
  '0.017*"word" + 0.017*"learning" + 0.016*"language" + 0.013*"learn" + '
  '0.010*"help" + 0.009*"reading" + 0.008*"new" + 0.008*"phrase" + '
  '0.008*"time" + 0.008*"grammar"'),
 (5,
  '0.040*"language" + 0.031*"word" + 0.025*"lea

In [7]:
print("hello world")

hello world


In [6]:
import pickle 
import pyLDAvis
import os 
import pyLDAvis.gensim


num_topics = 10

# Visualize the topics
pyLDAvis.enable_notebook()

#LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, termdocfreq, dictionary,sort_topics=False)
    

TypeError: drop() takes from 1 to 2 positional arguments but 3 were given

In [34]:
#coherence and score 
from gensim.models import CoherenceModel
# instantiate topic coherence model
import numpy as np
newcorpus = []
for lis in finallemma :
    for w in lis :
        if len(w)>2:
            newcorpus.append(w)

cm = CoherenceModel(model=lda_model, texts=feedtodict, dictionary=dictionary, coherence='c_v')
with np.errstate(invalid='ignore'):
    coherence_lda = cm.get_coherence()
    print('Coherence: ', coherence_lda)                
# getting topic coherence score 


#getting the perplexity
print('\nPerplexity: ', lda_model.log_perplexity(termdocfreq))

0.41130767176413513

Perplexity:  -5.128441183809427
